In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common import exceptions
from selenium.common.exceptions import NoSuchElementException
from openpyxl import load_workbook
from datetime import datetime
from selenium import webdriver
from time import sleep
import random
import re
import numpy as np
import pandas as pd

In [2]:
# Driver location
s = Service()#place of web driver"

# incognito
options = webdriver.ChromeOptions()
options.add_argument("--incognito")

# credential info
username = ""
pa_ss = ""

# URLs
login_url = 'https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin'

In [3]:
# Login
def login(driver):
    r1 = random.randint(3, 5)
    try:
        driver.get(login_url)
        sleep(r1)
        input_username = driver.find_element(By.ID, "username")
        input_password = driver.find_element(By.ID, "password")
        input_username.send_keys(username)
        sleep(r1)
        input_password.send_keys(pa_ss)
        sleep(r1)
        input_password.submit()
        sleep(r1)
    except:
        driver.get("https://www.linkedin.com/uas/login?session_redirect=https%3A%2F%2Fwww%2Elinkedin%2Ecom%2Fsearch%2Fresults%2Fpeople%2F%3Fkeywords%3Dengineer%26origin%3DFACETED_SEARCH%26sid%3DW%253B8&fromSignIn=true&trk=cold_join_sign_in")
        input_username = driver.find_element(By.ID, "username")
        input_password = driver.find_element(By.ID, "password")
        input_username.send_keys(username)
        input_password.send_keys(pa_ss)
        input_password.submit()
        sleep(r1)

In [4]:
url_done = []
distinct_total = []

In [5]:
def get_info_page(driver, url):
    r1 = random.randint(3, 5)
    r2 = random.randint(5, 10)
    sleep(r2)
    driver.get(url)
    driver.switch_to.window(driver.window_handles[0])
    links = driver.find_elements(By.XPATH, "//a[contains(@href, 'https://www.linkedin.com/in/')]")
    
    # read urls
    df = pd.read_excel('survey_info.xlsx', sheet_name = 'url', usecols = 'A')
    url_done = df.values.tolist()
    
    dist_url = []
    for link in links:
        href = link.get_attribute("href")
        this_page = any(href in x for x in dist_url)
        this_session = any(href in x for x in distinct_total)
        data_from_excel = any(href in x for x in url_done)
        if not this_page and not this_session and not data_from_excel:
            dist_url.append(href) # urls in a page
            distinct_total.append(href) # urls in a session
    
    for dis in dist_url:
        driver.execute_script("window.open('')")
        driver.switch_to.window(driver.window_handles[1])
        sleep(r1)
        driver.get(dis)

        #Open "Contact info" url
        driver.find_element(By.ID, "top-card-text-details-contact-info").click()
        sleep(r1)

        try:
            #name
            target_fullname = driver.find_element(By.ID, "pv-contact-info").text

            #email
            email = driver.find_element(By.XPATH, "//a[contains(@href, 'mailto:')]")
            href = email.get_attribute("href")
            split = href.split(":")

            #insert into list
            row = []
            row.extend([target_fullname, split[1]])
            print(row)
            contact_info.append(row)
        except:
            pass
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        sleep(r2)

In [6]:
def next_page(driver):
    r1 = random.randint(2, 6)
    r2 = random.randint(2, 6)
    action = ActionChains(driver)
    htmlelem = driver.find_element(By.TAG_NAME, "html")
    htmlelem.send_keys(Keys.END)
    sleep(r1)
    next_btn = driver.find_element(By.CSS_SELECTOR, "[aria-label='Next']")
    action.move_to_element(next_btn).click().perform()
    sleep(r2)

In [7]:
def close_all(data, url, place):
    df2 = pd.DataFrame(url)
    df3 = pd.DataFrame(data)
    with pd.ExcelWriter('survey_info.xlsx',engine="openpyxl", mode='a', if_sheet_exists='overlay') as writer:
        df2.to_excel(writer, sheet_name='url', startrow = writer.sheets['url'].max_row, header = False, index = False)
    with pd.ExcelWriter('survey_info.xlsx',engine="openpyxl", mode='a', if_sheet_exists='overlay') as writer:
        df3.to_excel(writer, sheet_name= place , startrow = writer.sheets[place].max_row, header = False, index = False)
    driver1.quit()

In [8]:
comp_url = 'https://www.linkedin.com/search/results/people/?geoUrn=%5B%22101355337%22%5D&keywords=IT&origin=FACETED_SEARCH&page=13&sid=wnY'
contact_info = []
driver1 = webdriver.Chrome(service = s, options=options)
driver1.set_window_size(750, 1000)
driver1.set_window_position(0,0, windowHandle='current')
login(driver1)

for i in range(0, 200):
    try:
        get_info_page(driver1, comp_url)
        next_page(driver1)
        comp_url = driver1.current_url
    except:
        break

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [9]:
close_all(contact_info, distinct_total, "Japan")

NameError: name 'driver1' is not defined

In [ ]:
print(len(contact_info))
print(len(distinct_total))
print(comp_url)